# Introduction: Agents.jl basics

This course introduces the [Agents.jl](https://juliadynamics.github.io/Agents.jl/stable/) Julia package for agent based modelling. The notebook is created by [Alejandra Ramirez](https://github.com/MA-Ramirez)
and George Datseris.

Agent based modelling are simulations where autonomous agents react to their environment and interact with each other given a predefined set of rules. 
These rules are formulated based on explicit statements rather than mathematical equations (such as: “If condition X is fulfilled, do action Y, and then perform operation Z on all nearby agents”).

_from Datseris & Parlitz. (2022)_

**Versions**: please ensure you have these packages installed with these or later versions:

In [63]:
import Pkg
Pkg.status(["Agents", "CairoMakie", "GLMakie", "InteractiveDynamics"])

Status `C:\Users\gd419\.julia\environments\v1.8\Project.toml`
  [46ada45e] Agents v5.8.0 `C:\Users\gd419\.julia\dev\Agents`
  [13f3f980] CairoMakie v0.10.2
  [e9467ef8] GLMakie v0.8.2
  [ec714cd0] InteractiveDynamics v0.22.1 `C:\Users\gd419\.julia\dev\InteractiveDynamics`


# Types in Julia

# Agents.jl basics

In this block we will go through a basic tutorial for Agents.jl. Another basic introduction can be found [online based on the Schelling model](https://juliadynamics.github.io/Agents.jl/stable/examples/schelling/).

## Simulations in Agents.jl

To set up an ABM simulation in Agents.jl, a user only needs to follow these steps:

1. Choose in what kind of space the agents will live in, for example a graph, a grid, etc. [Several spaces are provided by Agents.jl](https://juliadynamics.github.io/Agents.jl/stable/api/#Available-spaces-1) and can be initialized immediately.
2. Define the agent type (or types, for mixed models) that will populate the ABM. Agent types are standard Julia `mutable struct`s. They can be created manually, but typically you'd want to use [`@agent`](https://juliadynamics.github.io/Agents.jl/stable/tutorial/#.-The-agent-type(s)-1). The types must contain some mandatory fields, which is ensured by using `@agent`. The remaining fields of the agent type are up to user's choice.
3. The created agent type, the chosen space, optional additional model level properties, and other simulation tuning properties like schedulers or random number generators, are given to [`AgentBasedModel`](https://juliadynamics.github.io/Agents.jl/stable/tutorial/#Agents.AgentBasedModel). This instance defines the model within an Agents.jl simulation.
4. Provide functions that govern the time evolution of the ABM. A user can provide an agent-stepping function, that acts on each agent one by one, and/or a model-stepping function, that steps the entire model as a whole. These functions are standard Julia functions that take advantage of the [Agents.jl API](https://juliadynamics.github.io/Agents.jl/stable/api/). Once these functions are created, they are simply passed to [`step!`](https://juliadynamics.github.io/Agents.jl/stable/tutorial/#.-Evolving-the-model-1) to evolve the model.
5. _(Optional)_ [Visualize the model and animate its time evolution.](https://juliadynamics.github.io/Agents.jl/stable/agents_visualizations/) This can help checking that the model behaves as expected and there aren't any mistakes, or can be used in making figures for a paper/presentation.
6. Collect data. To do this, specify which data should be collected, by providing one standard Julia `Vector` of data-to-collect for agents, for example `[:mood, :wealth]`, and another one for the model. The agent data names are given as the keyword `adata` and the model as keyword `mdata` to the function [`run!`](https://juliadynamics.github.io/Agents.jl/stable/tutorial/#Agents.run!). This function outputs collected data in the form of a `DataFrame`.

## Predator-prey dynamics

We will now apply the tutorial on a predator-prey agent based model dynamics typically described by the Lokta-Volterra equations.

It describes the dynamics of biological systems in which two species interact, one as a predator and the other as a prey. Its rules are:

* The environment is a 2D grid.
* It contains Foxes 🦊, Rabbits 🐇 and Grass 🌱 (spatial property).
* 🦊 **---** eat **--->** 🐇 **---** eat **--->** 🌱
* 🦊, 🐇 can reproduce if they have enough energy, whie 🌱 constantly regrows
* 🦊, 🐇 perform random walks until they end up in a cell that has a food source (which they then eat)

The population will oscillate over time if the correct balance is achieved. If there is no balance, a population may become extinct.

In the following we will build this model in stages of increasing complexity!

## Rabbits hopping around

The first steps in any simulation with Agents.jl is to decide the space and the agent types. The space is a grid space so we define

In [65]:
using Agents

space = GridSpace((25, 25))

GridSpace with size (25, 25), metric=chebyshev, periodic=true

We'll now make an agent type for the 🐇. Actually, 🦊 and 🐇 have identical properties: 

- `energy`: represents an animal current energy level. If the energy drops below 0, the agent will die. Agents lose energy at each step of the simulation.
- `Δenergy`: controls how much energy is acquired after consuming a food source.
- `reproduction_prob`: reproduction probability. In this model individuals reproduce asexually *(assumption)*.

In addition to these properties, all agents must have an ID and a position property. This is made easy with the `@agent` macro, inheriting from the `GridAgent{2}`, the minimal agent that can be used in 2D grid space.

In [66]:
@agent Rabbit GridAgent{2} begin
    energy::Float64
    Δenergy::Float64
    reproduction_prob::Float64
end

This created a standard Julia composite type called `Rabbit` with the following properties:

In [67]:
for (name, type) in zip(fieldnames(Rabbit), fieldtypes(Rabbit))
    println(name, "::", type)
end

id::Int64
pos::Tuple{Int64, Int64}
energy::Float64
Δenergy::Float64
reproduction_prob::Float64


Okay, now that we have the space and the agent type, we just plug them into `AgentBasedModel`:

In [68]:
model = AgentBasedModel(Rabbit, space)

StandardABM with 0 agents of type Rabbit
 space: GridSpace with size (25, 25), metric=chebyshev, periodic=true
 scheduler: fastest

Alright! Perfect! But we need to also actually add some agents to this model! This is done with `add_agent!`. The simplest possible way to do this is by utilizing the automation offered by `add_agent!`: we can specify the extra properties of the agents, and then an agent is created with a correct ID and is added to a random position. For example:

In [69]:
energy = 10.0
Δenergy = 0.1
reproduction_prob = 0.1
add_agent!(model, energy, Δenergy, reproduction_prob)

Rabbit(1, (10, 5), 10.0, 0.1, 0.1)

This created, added to the model, and returned, a new agent.

It is always good practice to make the model using a function that accepts keyword arguments so let's do this, and in this function make a loop that creates the agents

In [70]:
using Random: Xoshiro # reproducibiilty

function init_rabbits(; total_agents = 100, griddims = (20, 20),
        energy = 10.0, Δenergy = 5.0, reproduction_prob = 0.1,
        seed = 1234,
    )
    space = GridSpace(griddims)
    rng = Xoshiro(seed)
    model = AgentBasedModel(Rabbit, space; rng)
    for n in 1:total_agents
        add_agent!(model, energy, Δenergy, reproduction_prob)
    end
    return model
end

model = init_rabbits()

StandardABM with 100 agents of type Rabbit
 space: GridSpace with size (20, 20), metric=chebyshev, periodic=true
 scheduler: fastest

There was one more thing we did: we initialized a dedicated **random number generator** (RNG) that we added to the model. This establishes reproducibility of our model, as this RNG will be used in all functions that involve randomness. We can also use this RNG ourselves:

In [71]:
rng = abmrng(model)
rand(rng, [0, 1])

0

Anyways, we now have a model with agents! The next step is to create the stepping (time evolution) rules! We only have an agent stepping function for now, which will random walk the agent, make it lose energy, and kill it if its energy drops below zero.

In [72]:
function rabbit_step!(rabbit, model)
    randomwalk!(rabbit, model, 1; ifempty = false) # radius 1
    rabbit.energy -= 1 # lose 1 energy per step
    if rabbit.energy ≤ 0
        kill_agent!(rabbit, model)
    end
end

rabbit_step! (generic function with 1 method)

Now, we can progress the model for one or more steps. In each step, the model will iterate through its agents and call this function on them. We progress the model using `step!`:

In [73]:
step!(model, rabbit_step!)
model

StandardABM with 100 agents of type Rabbit
 space: GridSpace with size (20, 20), metric=chebyshev, periodic=true
 scheduler: fastest

Or step for many steps:

In [74]:
step!(model, rabbit_step!, 10_000)
model

StandardABM with 0 agents of type Rabbit
 space: GridSpace with size (20, 20), metric=chebyshev, periodic=true
 scheduler: fastest

Alright, things seem to work. All rabbits die, since they eat no food but loose energy. But let's also visualize the model! To do so it is very easy! We choose a color/size/marker for the agents and call `abmplot`!

In [75]:
using InteractiveDynamics # where plotting comes from

using GLMakie # this allows videos and interactive apps
model = init_rabbits() # refresh model
rabbitcolor(a) = :orange
rabbitmarker(a) = :circle
fig, = abmplot(model; ac = rabbitcolor, am = rabbitmarker)
fig

giving the stepping functions to the `abmplot` function will allow use to interactive evolve the model!

In [76]:
fig, = abmplot(model; 
    ac = rabbitcolor, am = rabbitmarker,
    agent_step! = rabbit_step!,    
)

fig

## Spring time! (grass dynamics, reproduction dynamics)

Let's now add grass dynamics, which will introduce more complexity to the model by

- adding a model properties that are spatial fields
- accessing model properties
- adding a model step function
- Allowing rabbits to generate more rabbits once they have enough energy

To do this, we first modify the model creation function to add two properties for grass: one that checks if it is fully grown (`Bool`) and a countdown to regrowth (`Int`). They are spatial fields because each cell has its own grass counter!

In [158]:
function init_rabbits_grass(; 
        # Rabbit properties:
        energy = 10.0, Δenergy = 5.0, reproduction_prob = 0.1,
        # Grass properties:
        griddims = (20, 20), regrowth_time = 10,
        # General:
        seed = 1234, total_agents = 100, 
    )
    space = GridSpace(griddims)
    rng = Xoshiro(seed)
    # Initialize grass properties
    properties = (
        fully_grown = falses(griddims),
        countdown = rand(rng, 1:regrowth_time, griddims),
        regrowth_time = regrowth_time,
    )
    
    model = AgentBasedModel(Rabbit, space; rng, properties)
    
    for n in 1:total_agents
        add_agent!(model, energy, Δenergy, reproduction_prob)
    end
    return model
end

model = init_rabbits_grass()

StandardABM with 100 agents of type Rabbit
 space: GridSpace with size (20, 20), metric=chebyshev, periodic=true
 scheduler: fastest
 properties: fully_grown, countdown, regrowth_time

The grass dynamics are govered by a _model stepping function_ that accepts a single argument (the model) and progresses it by one step. Here, this function is about the grass dynamics

In [159]:
function grass_step!(model)
    # iterate over all positions (cells) in the model
    # they are 2-tuples of integers
    for p in positions(model)
        if !model.fully_grown[p[1], p[2]]
            model.countdown[p[1], p[2]] -=1
            if model.countdown[p[1], p[2]] ≤ 0
                model.fully_grown[p[1], p[2]] = true
                model.countdown[p[1], p[2]] = model.regrowth_time
            end
        end
    end
end

grass_step! (generic function with 1 method)

We utilized a nice feature of Agents.jl above: model properties can be accessed with the `.property` syntax, as if they were fields of the model type (they aren't, but Julia allows such nice things ;) ).

We also modify the rabbit dynamics to eat and generate offsprings

In [160]:
function rabbit_step!(rabbit, model)
    randomwalk!(rabbit, model, 1; ifempty = false) # radius 1
    rabbit.energy -= 1 # lose 1 energy per step
    if rabbit.energy ≤ 0
        kill_agent!(rabbit, model)
    end
    # consume a food source if available.
    eat_grass!(rabbit, model)
    # otherwise, it lives and reproduces with some probability.
    if rand(abmrng(model)) ≤ rabbit.reproduction_prob
        reproduce!(rabbit, model)
    end
end

function eat_grass!(rabbit, model)
    # if grass is grown, eat it to gain energy
    if model.fully_grown[rabbit.pos...]
        rabbit.energy += rabbit.Δenergy
        model.fully_grown[rabbit.pos...] = false
        model.countdown[rabbit.pos...] = model.regrowth_time
    end
    return
end

function reproduce!(agent::A, model) where {A<:AbstractAgent}
    agent.energy /= 2
    # generate a new agent of type `A`
    id = nextid(model)
    offspring = A(id, agent.pos, agent.energy, agent.reproduction_prob, agent.Δenergy)
    add_agent_pos!(offspring, model)
    return
end

reproduce! (generic function with 2 methods)

Alright, so we have now can do:

In [161]:
model = init_rabbits_grass()

StandardABM with 100 agents of type Rabbit
 space: GridSpace with size (20, 20), metric=chebyshev, periodic=true
 scheduler: fastest
 properties: fully_grown, countdown, regrowth_time

In [162]:
step!(model, rabbit_step!, grass_step!, 100)
model

StandardABM with 123 agents of type Rabbit
 space: GridSpace with size (20, 20), metric=chebyshev, periodic=true
 scheduler: fastest
 properties: fully_grown, countdown, regrowth_time

Agents seem to reduce in numbers. Let's visualize again to see how the model behaves. We can learn do one more thing during visualization: visualize a spatial property as a heatmap! This is as easy as choosing the `heatarray` argument to be a model property, or a function of the model, like so:

In [163]:
model = init_rabbits_grass()

grasscolor(model) = 1 .- (model.countdown ./ model.regrowth_time)
heatkwargs = (colormap = [:brown, :green], colorrange = (0,1))

plotkwargs = (;
    ac = "white",
    as = 25,
    #additional keyword arguments propagated to the scatter! call.
    scatterkwargs = (strokewidth = 1.0, strokecolor = :black),
    heatarray = grasscolor,
    heatkwargs = heatkwargs,
)

fig, = abmplot(model; 
    agent_step! = rabbit_step!, model_step! = grass_step!,
    plotkwargs...,
)

fig

In [164]:
GLMakie.activate!()

## Hungry hungry foxes!

Let's now add foxes into the model, which will introduce even more complexity to the model by

- adding another agent "type" (not really, see below)
- adding neighboring agent searches
- adding two different agent stepping functions depending on the agent "type"
- agent-dependent plotting of agents

Rabbits and foxes have indentical _property types_, even if they have different _values_ for the properties. Normally, we would have to make different agent types for the rabbits and the foxes, but here we can cheat the system by doing:

In [165]:
@agent Animal GridAgent{2} begin
    type::Symbol # either `:Rabbit` or `:Fox`
    energy::Float64
    Δenergy::Float64
    reproduction_prob::Float64
end

and we'll differentiate between foxes and rabbits on the basis of the animal type. This means that we reform the agent stepping function into:

In [179]:
function animal_step!(animal, model)
    if animal.type == :Rabbit
        rabbit_step!(animal, model)
    elseif animal.type == :Fox
        fox_step!(animal, model)
    end
end

animal_step! (generic function with 1 method)

and we reuse the `rabbit_step!` function from above, but we define a new function for foxes. Foxes are agile and adept hunters of rabbits and therefore, unlike rabbits, they can look for food in a neighborhood around them instead of only the position they are at. We achieve this using the `nearby_agents` function:

In [180]:
function fox_step!(fox, model)
    # move
    randomwalk!(fox, model, 1; ifempty=false)
    # die
    fox.energy -= 1
    if fox.energy < 0
        kill_agent!(fox, model)
        return
    end
    # eat 
    eat_rabbit!(fox, model)
    # reproduce
    if rand(abmrng(model)) <= fox.reproduction_prob
        reproduce!(fox, model)
    end
    return nothing
end

function eat_rabbit!(fox, model)
    # attempt to find a neary rabbit
    for animal in nearby_agents(fox, model)
        if animal.type == :Rabbit # found one!
            kill_agent!(animal, model)
            fox.energy += fox.Δenergy
        end
    end
    return
end            

eat_rabbit! (generic function with 1 method)

Don't forget to update the `reproduce!` function to take into account this new extra field `type`!

In [181]:
function reproduce!(agent::A, model) where {A<:AbstractAgent}
    agent.energy /= 2
    # generate a new agent of type `A`
    id = nextid(model)
    offspring = A(id, agent.pos, agent.type, agent.energy, agent.reproduction_prob, agent.Δenergy)
    add_agent_pos!(offspring, model)
    return
end

reproduce! (generic function with 2 methods)

In [182]:
Don't 

UndefVarError: UndefVarError: Don not defined

Okay, now let's make a function that creates a model that has both rabbits and foxes!

In [192]:
function init_foxes_rabbits_grass(; 
        # Rabbit properties:
        total_rabbits = 100, 
        rabbit_energy = 10.0, rabbit_Δenergy = 5.0, rabbit_r_prob = 0.2,
        # Fox properties:
        total_foxes = 10,
        fox_energy = 20.0, fox_Δenergy = 10.0, fox_r_prob = 0.1,
        # Grass properties:
        griddims = (20, 20), regrowth_time = 10,
        # General:
        seed = 1234, total_agents = 100, 
    )
    space = GridSpace(griddims)
    rng = Xoshiro(seed)
    # Initialize grass properties
    properties = (
        fully_grown = falses(griddims),
        countdown = rand(rng, 1:regrowth_time, griddims),
        regrowth_time = regrowth_time,
    )

    # Use a scheduler that makes rabbits act first
    scheduler = Schedulers.ByProperty(animal -> animal.type == :Rabbit ? 1 : 0)
    model = AgentBasedModel(Animal, space; rng, properties, scheduler)

    for n in 1:total_rabbits
        add_agent!(model, :Rabbit, rabbit_energy, rabbit_Δenergy, rabbit_r_prob)
    end
    for n in 1:total_foxes
        add_agent!(model, :Fox, fox_energy, fox_Δenergy, fox_r_prob)
    end
    return model
end

model = init_foxes_rabbits_grass()

StandardABM with 110 agents of type Animal
 space: GridSpace with size (20, 20), metric=chebyshev, periodic=true
 scheduler: Agents.Schedulers.ByProperty{var"#47#48"}
 properties: fully_grown, countdown, regrowth_time

Notice that we also for the first time changed the model _scheduler_, which schedules which agents act first.

In [195]:
model = init_foxes_rabbits_grass()
step!(model, animal_step!, grass_step!)
model

StandardABM with 106 agents of type Animal
 space: GridSpace with size (20, 20), metric=chebyshev, periodic=true
 scheduler: Agents.Schedulers.ByProperty{var"#47#48"}
 properties: fully_grown, countdown, regrowth_time

Let's now visualize the time evolution for this model, using a different color and marker for rabbits and foxes

In [196]:
model = init_foxes_rabbits_grass()

grasscolor(model) = 1 .- (model.countdown ./ model.regrowth_time)
heatkwargs = (colormap = [:brown, :green], colorrange = (0,1))

animalcolor(animal) = animal.type == :Rabbit ? "white" : "orange"
animalmarker(animal) = animal.type == :Rabbit ? :circle : :dtriangle

plotkwargs = (;
    ac = animalcolor,
    am = animalmarker,
    as = 25,
    #additional keyword arguments propagated to the scatter! call.
    scatterkwargs = (strokewidth = 1.0, strokecolor = :black),
    heatarray = grasscolor,
    heatkwargs = heatkwargs,
)

fig, = abmplot(model; 
    agent_step! = animal_step!, model_step! = grass_step!,
    plotkwargs...,
)

fig

# Collect data

6. **Collect data**.

To collect data, we should specify the data we want to collect by providing one standard Julia `Vector`of data-to-collect for agents. Running the model and collecting data while the model runs is done with the `run!` function. The agent data names are given as the keyword `adata` and the model as keyword `mdata` to the function `run!`. This function outputs collected data in the form of a `DataFrame`.

`run!(model, agent_step!, model_step!, n::Function; kwargs...) → agent_df, model_df`
* `adata::Vector` means "agent data to collect". It specificies which properties of the agents we would like to collect as data e.g. `[:age, :height]`. If an entry is a function, e.g. `f`, then the data for this entry is `f(a)`for each agent `a`. `[:age, :height, :f]`
* `adata::Vector{<:Tuple}`: if `adata` is a vector of tuples, data aggregation is done over the agent properties. For each 2-tuple, the first entry is the "key" (any entry like the ones mentioned above, e.g. `:weight, `). The second entry is an aggregating function that aggregates the key, e.g. `mean`,`maximum`. So, continuing from the above example, we would have `adata = [(:weight, mean), (f, maximum)]`


*Additional useful functions:*
* `paramscan`: it performs data collection while scanning ranges of the parameters of the model.
* `ensemblerun!`: performs ensemble simulations and data collection.

In [97]:
# We will count the number of Rabbits, Foxes and fully-grown Grass for the time evolution of the system
rabbit(a) = a isa Rabbit
fox(a) = a isa Fox
count_grass(model) = count(model.fully_grown)

count_grass (generic function with 1 method)

In [98]:
foxrabbitgrass = initialize_model()
steps = 1000
adata = [(rabbit, count), (fox, count)]
mdata = [count_grass]
adf, mdf = run!(foxrabbitgrass, rabbitfox_step!, grass_step!, steps; adata, mdata)

UndefVarError: UndefVarError: MersenneTwister not defined

We will now plot the evolution of the system

In [99]:
function plot_population_timeseries(adf, mdf)
    figure = Figure(resolution = (600, 400))
    ax = figure[1, 1] = Axis(figure; xlabel = "Step", ylabel = "Population")
    rabbitl = lines!(ax, adf.step, adf.count_rabbit, color = :cornsilk4)
    foxl = lines!(ax, adf.step, adf.count_fox, color =  RGBAf(1.0, 0.76, 0.3))
    grassl = lines!(ax, mdf.step, mdf.count_grass, color = :green)
    figure[1, 2] = Legend(figure, [rabbitl, foxl, grassl], ["Rabbits", "Foxes", "Grass"])
    figure
end

plot_population_timeseries(adf, mdf)

UndefVarError: UndefVarError: adf not defined

The plot shows the population dynamics over time. Initially, foxes become extinct because they consume the rabbits too quickly. The few remaining rabbits reproduce and gradually reach an equilibrium that can be supported by the amount of available grass.

## Exercises

For the exercises, you will be coding the agent-based simulation for different models.

To set up an ABM simulation in Agents.jl, we advice to use the following steps:

1. **Choose** the kind of **space** that the agents will live in.

2. **Define** the **agent type** (or types, for mixed models) of the ABM.

3. **Define** the **model** within an Agents.jl simulation by using its universal structure `AgentBasedModel`.

4. Provide the **time evolution functions** of the ABM.

5. **Visualise** the model and animate its time evolution.

6. **Collect data**.

*Note:* Step 5 and 6 can be interchangeable depending on the simulation's objective.

### 1. Wright-Fisher model

### Neutral model
Create an agent-based model for the neutral Wright-Fisher model. This population genetics model describes the evolution of a fixed size population that undergoes genetic drift. 

* The model will have no spatial property. Therefore, the agents are defined as `NoSpaceAgent`.
* The population is composed of `n` haploid individuals. The main property of each agent is `trait`, which will represent the fitness of the agent on a range from 0 to 1.
* The agents will be activated randomly in the model. Therefore, we need to seed the simulation to make it reproducible. In addition, we would like for our model to be initialized with the property `population_size`, in order to freely modify the amount of individuals in the population.
* At each generation, `n` offsprings replace the parents. This means that `n` offspring are sampled with replacement from the parent generation.
* The objective of the simulation is to plot the evolution of the average trait over time.

### Model with selection
Create an agent-based model for the Wright-Fisher model with selection. This means that the model now samples individuals according to their trait values (we assume that the inviduals' fitness is correlated to their trait values).

-----
*Tip: use the function `sample!`from Agents.jl*

`sample!(model::ABM, n [, weight]; kwargs...)`

It replaces the agents of the model with a random sample of the current agents with size `n`.
* `weight`: Symbol (agent field) or function (input agent out put number) to weight the sampling. This means that the higher the weight of the agent, the higher the probability that this agent will be chosen in the new sampling.

### 2. Spatial Rock-Paper-Scissors

Create an agent-based model for the spatial rock-paper-scissors game.

The agent-based model simulation describes the evolution of 3 competing strategies ("Rock", "Paper" and "Scissors") that interact through cyclic, nonhierarchical interactions.
To be more precise, the interactions follow a Rock-Paper-Scissors construction:
* Rock + Scissors --> Rock
* Scissors + Paper --> Scissors
* Paper + Rock --> Paper

Agents interact with their nearest 4 neighbours through selection or reproduction, both of which reactions occur as Poisson processes at rates $\sigma$ and $\mu$.

In addition, the model explores the effect of mobility on the diversity of the population via the exchange rate $\epsilon$.

* Selection reflects cyclic dominance: Rock --> Scissors --> Paper --> Rock
* Reproduction of strategies is only allowed on empty neighbouring sites, to mimic a finite carrying capacity of the system
* Mobility is represented via $\epsilon$, this exchange rate represent the likelihood of agents to swap position with a neighbouring individual or hop onto an empty neighbouring site.

![Spatial RPS instructions](SpatialRPS_instructions.png)

Whether selection, reproduction or mobility occurs is computed according to the reaction rates using the [Gillespie algorithm](https://en.wikipedia.org/wiki/Gillespie_algorithm).

Explore the system's evolution for different exchange rate values.

